# IE 582 Project - Group ID 10

In [306]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn import preprocessing
import statistics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.linear_model import  Lasso
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.tree import DecisionTreeRegressor

148 - England Premier League ID

In [659]:
matches = pd.read_csv('matches.csv')
matches = matches.drop(['epoch', 'match_status', 'match_live', 'match_hometeam_penalty_score' , 'match_awayteam_penalty_score',  'match_hometeam_extra_score', 'match_awayteam_extra_score'], axis=1)
matches.columns = ['away_id', 'home_id', 'match_id', 'home_name', 'away_name' ,  'home_score', 'away_score', 'home_half_score', 'away_half_score', 'league_id']
matches.head()

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id
0,7109,7097,41196,Levante,Valencia,1.0,1.0,1.0,1.0,468
1,2614,2619,13331,Crystal Palace,Southampton,0.0,1.0,0.0,1.0,148
2,3224,3238,17683,Eintracht Frankfurt,FC Augsburg,1.0,2.0,0.0,1.0,195
3,3235,3223,17684,SV Werder Bremen,Schalke,1.0,2.0,1.0,1.0,195
4,3237,3225,17682,Bayern Munich,1. FSV Mainz 05,4.0,0.0,2.0,0.0,195


In [660]:
matches.fillna('', inplace = True) 

In [661]:
matches[matches['league_id'] == 148].tail(20)

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id
5692,2616,2612,273254,Everton,Chelsea,3,1,1,0,148
5702,2619,2623,273259,Watford,Crystal Palace,0,0,0,0,148
5703,2621,2615,273252,Bournemouth,Liverpool,0,3,0,2,148
5705,2629,2628,273258,Tottenham,Burnley,5,0,3,0,148
5719,2627,2626,273255,Manchester City,Manchester Utd,1,2,0,2,148
5745,2611,2632,273251,Aston Villa,Leicester,1,4,1,2,148
5746,2614,2630,273256,Newcastle,Southampton,2,1,0,0,148
5747,2654,2641,273257,Norwich,Sheffield Utd,1,2,1,0,148
5758,2646,2613,273253,Brighton,Wolves,2,2,2,2,148
5766,2617,2620,273260,West Ham,Arsenal,,,,,148


**Let's just extract only the matches from English Premier League**

In [662]:
matches_gb = matches[matches['league_id'] == 148].copy()
matches_gb = matches_gb.reset_index(drop=True)

In [663]:
matches_gb['home_id'] = pd.Categorical(matches_gb['home_id'])
dfDummies = pd.get_dummies(matches_gb['home_id'], prefix = 'home_id')
matches_gb = pd.concat([matches_gb, dfDummies], axis=1)

matches_gb['away_id'] = pd.Categorical(matches_gb['away_id'])
dfDummies = pd.get_dummies(matches_gb['away_id'], prefix = 'away_id')
matches_gb = pd.concat([matches_gb, dfDummies], axis=1)

In [664]:
matches_gb.tail(12)

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id,...,away_id_2623,away_id_2626,away_id_2627,away_id_2628,away_id_2629,away_id_2630,away_id_2632,away_id_2641,away_id_2646,away_id_2654
700,2646,2613,273253,Brighton,Wolves,2,2,2,2,148,...,0,0,0,0,0,0,0,0,1,0
701,2617,2620,273260,West Ham,Arsenal,,,,,148,...,0,0,0,0,0,0,0,0,0,0
702,2623,2621,273266,Liverpool,Watford,,,,,148,...,1,0,0,0,0,0,0,0,0,0
703,2615,2616,273263,Chelsea,Bournemouth,,,,,148,...,0,0,0,0,0,0,0,0,0,0
704,2630,2629,273262,Burnley,Newcastle,,,,,148,...,0,0,0,0,0,1,0,0,0,0
705,2632,2654,273268,Sheffield Utd,Aston Villa,,,,,148,...,0,0,0,0,0,0,1,0,0,0
706,2641,2611,273265,Leicester,Norwich,,,,,148,...,0,0,0,0,0,0,0,1,0,0
707,2620,2614,273269,Southampton,West Ham,,,,,148,...,0,0,0,0,0,0,0,0,0,0
708,2612,2627,273267,Manchester Utd,Everton,,,,,148,...,0,0,0,0,0,0,0,0,0,0
709,2628,2646,273270,Wolves,Tottenham,,,,,148,...,0,0,0,1,0,0,0,0,0,0


In [665]:
cut = matches_gb[matches_gb['home_score'] == ''].index[0]
coming_games = matches_gb.iloc[cut:,:].copy().reset_index(drop=True)
coming_games

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id,...,away_id_2623,away_id_2626,away_id_2627,away_id_2628,away_id_2629,away_id_2630,away_id_2632,away_id_2641,away_id_2646,away_id_2654
0,2617,2620,273260,West Ham,Arsenal,,,,,148,...,0,0,0,0,0,0,0,0,0,0
1,2623,2621,273266,Liverpool,Watford,,,,,148,...,1,0,0,0,0,0,0,0,0,0
2,2615,2616,273263,Chelsea,Bournemouth,,,,,148,...,0,0,0,0,0,0,0,0,0,0
3,2630,2629,273262,Burnley,Newcastle,,,,,148,...,0,0,0,0,0,1,0,0,0,0
4,2632,2654,273268,Sheffield Utd,Aston Villa,,,,,148,...,0,0,0,0,0,0,1,0,0,0
5,2641,2611,273265,Leicester,Norwich,,,,,148,...,0,0,0,0,0,0,0,1,0,0
6,2620,2614,273269,Southampton,West Ham,,,,,148,...,0,0,0,0,0,0,0,0,0,0
7,2612,2627,273267,Manchester Utd,Everton,,,,,148,...,0,0,0,0,0,0,0,0,0,0
8,2628,2646,273270,Wolves,Tottenham,,,,,148,...,0,0,0,1,0,0,0,0,0,0
9,2626,2617,273261,Arsenal,Manchester City,,,,,148,...,0,1,0,0,0,0,0,0,0,0


In [666]:
matches_gb = matches_gb.iloc[:cut,:]

In [667]:
matches_gb.tail()

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id,...,away_id_2623,away_id_2626,away_id_2627,away_id_2628,away_id_2629,away_id_2630,away_id_2632,away_id_2641,away_id_2646,away_id_2654
696,2627,2626,273255,Manchester City,Manchester Utd,1,2,0,2,148,...,0,0,1,0,0,0,0,0,0,0
697,2611,2632,273251,Aston Villa,Leicester,1,4,1,2,148,...,0,0,0,0,0,0,0,0,0,0
698,2614,2630,273256,Newcastle,Southampton,2,1,0,0,148,...,0,0,0,0,0,0,0,0,0,0
699,2654,2641,273257,Norwich,Sheffield Utd,1,2,1,0,148,...,0,0,0,0,0,0,0,0,0,1
700,2646,2613,273253,Brighton,Wolves,2,2,2,2,148,...,0,0,0,0,0,0,0,0,1,0


**Now let's add a column to indicate the winner, or tie in case there is no winner:**

In [668]:
homes = matches_gb.iloc[:,5:6].values.tolist()
aways = matches_gb.iloc[:,6:7].values.tolist()
away_teams = matches_gb.iloc[:,0:1].values.tolist()
home_teams = matches_gb.iloc[:,1:2].values.tolist()

In [669]:
winners = list()
for i in range(len(homes)):
    if homes[i][0] > aways[i][0]:
        winners.append(str(home_teams[i][0]))
        
    elif homes[i][0] == aways[i][0]:
        winners.append("Tie")
        
    elif homes[i][0] < aways[i][0]:
        winners.append(winners.append(str(away_teams[i][0])))

In [670]:
for i in winners:
    if i == None:
        winners.remove(i)

In [671]:
matches_gb['winners'] = winners

In [672]:
matches_gb.tail(10)

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id,...,away_id_2626,away_id_2627,away_id_2628,away_id_2629,away_id_2630,away_id_2632,away_id_2641,away_id_2646,away_id_2654,winners
691,2613,2617,273241,Arsenal,Brighton,1,2,0,1,148,...,0,0,0,0,0,0,0,0,0,2613
692,2616,2612,273254,Everton,Chelsea,3,1,1,0,148,...,0,0,0,0,0,0,0,0,0,2612
693,2619,2623,273259,Watford,Crystal Palace,0,0,0,0,148,...,0,0,0,0,0,0,0,0,0,Tie
694,2621,2615,273252,Bournemouth,Liverpool,0,3,0,2,148,...,0,0,0,0,0,0,0,0,0,2621
695,2629,2628,273258,Tottenham,Burnley,5,0,3,0,148,...,0,0,0,1,0,0,0,0,0,2628
696,2627,2626,273255,Manchester City,Manchester Utd,1,2,0,2,148,...,0,1,0,0,0,0,0,0,0,2627
697,2611,2632,273251,Aston Villa,Leicester,1,4,1,2,148,...,0,0,0,0,0,0,0,0,0,2611
698,2614,2630,273256,Newcastle,Southampton,2,1,0,0,148,...,0,0,0,0,0,0,0,0,0,2630
699,2654,2641,273257,Norwich,Sheffield Utd,1,2,1,0,148,...,0,0,0,0,0,0,0,0,1,2654
700,2646,2613,273253,Brighton,Wolves,2,2,2,2,148,...,0,0,0,0,0,0,0,1,0,Tie


**I would like our model to know the stronger teams, so I would create a new function to calculate each teams points by number of wins and scale these points between 1 to 10**

In [673]:
teams = matches_gb.home_id.unique()

In [674]:
teams_list = matches_gb.home_id.unique().astype(str).tolist()

In [675]:
def calculate_team_points(match_id):
    team_points = dict()
    for i in teams_list:
        team_points[i] = 0
        
    index = index = matches_gb[matches_gb['match_id'] == match_id ].index[0]
    games = matches_gb.loc[:index,].copy()
    winners2 = games.winners.tolist()
    
    for i in teams_list:
        team_points[i] = winners.count(i)
    
    table = pd.DataFrame(zip(team_points.keys(), team_points.values()), columns = ["teams", 'points'])
    table = table.sort_values(by = 'points', ascending= False).reset_index(drop=True)
    
    x = table[['points']].values.astype(float)
    
    
    # Scaling points between 1 - 10
    news = list()
    for k in x:
        rng = max(x)[0] - min(x)[0] 
        new = 1 + abs(k -  min(x)[0])*(9/rng)
        news.append(new[0])
        
   
    team_points_test = table.copy()
    team_points_test['points'] = news
    return team_points_test
    

In [676]:
stats = pd.read_csv('stats.csv')
stats.head()

,match_id,home_BallPossession,home_CornerKicks,home_Fouls,home_GoalAttempts,home_GoalkeeperSaves,home_Offsides,home_ShotsoffGoal,home_ShotsonGoal,home_YellowCards,...,home_DistanceCovered(metres),away_DistanceCovered(metres),home_PassSuccess%,away_PassSuccess%,home_Attacks,home_DangerousAttacks,away_Attacks,away_DangerousAttacks,home_,away_
0,13327,71%,12.0,7.0,35.0,3.0,1.0,14.0,9.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13329,33%,3.0,8.0,7.0,4.0,2.0,5.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13331,45%,5.0,14.0,14.0,3.0,0.0,9.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13446,51%,7.0,7.0,6.0,3.0,0.0,4.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13447,49%,3.0,18.0,20.0,1.0,1.0,8.0,6.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Let's just extract the features we need:

In [677]:
stats = stats.iloc[:,[0, 1,7,8,10,16,17,28,30]].copy()

In [678]:
bets = pd.read_csv('bets.csv')
bets = bets[(bets.variable == 'odd_1') | (bets.variable == 'odd_2') | (bets.variable == 'odd_x')]

In [679]:
bets_table = pd.pivot_table(bets.loc[:,['match_id', 'variable','value']], index=['match_id'], columns='variable')
bets_table['match_id'] = bets_table.index
bets_table = bets_table.reset_index(drop=True)
bets_table.head(10)


value                     match_id
variable     odd_1     odd_2     odd_x         
0         6.039825  1.387018  4.721579   145899
1         2.109828  2.843966  3.616724   145900
2         1.960000  3.650000  3.490000   146845
3         1.995714  3.397143  3.092857   147984
4         2.135294  3.815294  3.245294   147990
5         1.920588  4.316471  3.483529   147991
6         8.396471  1.395882  4.920588   147992
7         8.856471  1.406471  4.657647   147993
8         2.904118  2.427059  3.509412   147995
9         1.765294  5.165294  3.592941   147996

In [680]:
merged = pd.merge(matches_gb, bets_table, on='match_id').rename({'(value,odd_1)':'odd_1'})

c:\users\gunay.eser\python\python36\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [681]:
merged.columns

Index([         'away_id',          'home_id',         'match_id',
              'home_name',        'away_name',       'home_score',
             'away_score',  'home_half_score',  'away_half_score',
              'league_id',     'home_id_2611',     'home_id_2612',
           'home_id_2613',     'home_id_2614',     'home_id_2615',
           'home_id_2616',     'home_id_2617',     'home_id_2619',
           'home_id_2620',     'home_id_2621',     'home_id_2623',
           'home_id_2626',     'home_id_2627',     'home_id_2628',
           'home_id_2629',     'home_id_2630',     'home_id_2632',
           'home_id_2641',     'home_id_2646',     'home_id_2654',
           'away_id_2611',     'away_id_2612',     'away_id_2613',
           'away_id_2614',     'away_id_2615',     'away_id_2616',
           'away_id_2617',     'away_id_2619',     'away_id_2620',
           'away_id_2621',     'away_id_2623',     'away_id_2626',
           'away_id_2627',     'away_id_2628',     'away_id_26

In [682]:
merged.columns = ['away_id',          'home_id',         'match_id',
              'home_name',        'away_name',       'home_score',
             'away_score',  'home_half_score',  'away_half_score',
              'league_id',     'home_id_2611',     'home_id_2612',
           'home_id_2613',     'home_id_2614',     'home_id_2615',
           'home_id_2616',     'home_id_2617',     'home_id_2619',
           'home_id_2620',     'home_id_2621',     'home_id_2623',
           'home_id_2626',     'home_id_2627',     'home_id_2628',
           'home_id_2629',     'home_id_2630',     'home_id_2632',
           'home_id_2641',     'home_id_2646',     'home_id_2654',
           'away_id_2611',     'away_id_2612',     'away_id_2613',
           'away_id_2614',     'away_id_2615',     'away_id_2616',
           'away_id_2617',     'away_id_2619',     'away_id_2620',
           'away_id_2621',     'away_id_2623',     'away_id_2626',
           'away_id_2627',     'away_id_2628',     'away_id_2629',
           'away_id_2630',     'away_id_2632',     'away_id_2641',
           'away_id_2646',     'away_id_2654',          'winners', 'odd_1',
       'odd_2', 'odd_x']

In [683]:
merged['odd_1'] = 1/merged['odd_1']
merged['odd_2'] = 1/merged['odd_2']
merged['odd_x'] = 1/merged['odd_x']
merged.head()

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id,...,away_id_2629,away_id_2630,away_id_2632,away_id_2641,away_id_2646,away_id_2654,winners,odd_1,odd_2,odd_x
0,2612,2616,150842,Chelsea,Everton,0,0,0,0,148,...,0,0,0,0,0,0,Tie,0.194175,0.568182,0.270270
1,2627,2626,150844,Manchester City,Manchester Utd,3,1,1,0,148,...,0,0,0,0,0,0,2626,0.343249,0.410959,0.303030
2,2615,2626,156848,Manchester City,Bournemouth,3,1,1,1,148,...,0,0,0,0,0,0,2626,0.370430,0.375756,0.340690
3,2623,2611,156847,Leicester,Watford,2,0,2,0,148,...,0,0,0,0,0,0,2611,0.588560,0.202962,0.289960
4,2629,2619,156845,Crystal Palace,Burnley,2,0,1,0,148,...,1,0,0,0,0,0,2619,0.432232,0.305744,0.345338


In [684]:
merged.columns

Index(['away_id', 'home_id', 'match_id', 'home_name', 'away_name',
       'home_score', 'away_score', 'home_half_score', 'away_half_score',
       'league_id', 'home_id_2611', 'home_id_2612', 'home_id_2613',
       'home_id_2614', 'home_id_2615', 'home_id_2616', 'home_id_2617',
       'home_id_2619', 'home_id_2620', 'home_id_2621', 'home_id_2623',
       'home_id_2626', 'home_id_2627', 'home_id_2628', 'home_id_2629',
       'home_id_2630', 'home_id_2632', 'home_id_2641', 'home_id_2646',
       'home_id_2654', 'away_id_2611', 'away_id_2612', 'away_id_2613',
       'away_id_2614', 'away_id_2615', 'away_id_2616', 'away_id_2617',
       'away_id_2619', 'away_id_2620', 'away_id_2621', 'away_id_2623',
       'away_id_2626', 'away_id_2627', 'away_id_2628', 'away_id_2629',
       'away_id_2630', 'away_id_2632', 'away_id_2641', 'away_id_2646',
       'away_id_2654', 'winners', 'odd_1', 'odd_2', 'odd_x'],
      dtype='object')

In [685]:
matches_stats = pd.merge(merged[['winners','match_id','home_id', 'away_id', 'odd_1', 'odd_2', 'odd_x', 'home_id_2611', 'home_id_2612', 'home_id_2613',
       'home_id_2614', 'home_id_2615', 'home_id_2616', 'home_id_2617',
       'home_id_2619', 'home_id_2620', 'home_id_2621', 'home_id_2623',
       'home_id_2626', 'home_id_2627', 'home_id_2628', 'home_id_2629',
       'home_id_2630', 'home_id_2632', 'home_id_2641', 'home_id_2646',
       'home_id_2654', 'away_id_2611', 'away_id_2612', 'away_id_2613',
       'away_id_2614', 'away_id_2615', 'away_id_2616', 'away_id_2617',
       'away_id_2619', 'away_id_2620', 'away_id_2621', 'away_id_2623',
       'away_id_2626', 'away_id_2627', 'away_id_2628', 'away_id_2629',
       'away_id_2630', 'away_id_2632', 'away_id_2641', 'away_id_2646',
       'away_id_2654'] ],stats, on='match_id')# .isnull().sum()

In [686]:
matches_stats.columns

Index(['winners', 'match_id', 'home_id', 'away_id', 'odd_1', 'odd_2', 'odd_x',
       'home_id_2611', 'home_id_2612', 'home_id_2613', 'home_id_2614',
       'home_id_2615', 'home_id_2616', 'home_id_2617', 'home_id_2619',
       'home_id_2620', 'home_id_2621', 'home_id_2623', 'home_id_2626',
       'home_id_2627', 'home_id_2628', 'home_id_2629', 'home_id_2630',
       'home_id_2632', 'home_id_2641', 'home_id_2646', 'home_id_2654',
       'away_id_2611', 'away_id_2612', 'away_id_2613', 'away_id_2614',
       'away_id_2615', 'away_id_2616', 'away_id_2617', 'away_id_2619',
       'away_id_2620', 'away_id_2621', 'away_id_2623', 'away_id_2626',
       'away_id_2627', 'away_id_2628', 'away_id_2629', 'away_id_2630',
       'away_id_2632', 'away_id_2641', 'away_id_2646', 'away_id_2654',
       'home_BallPossession', 'home_ShotsoffGoal', 'home_ShotsonGoal',
       'away_BallPossession', 'away_ShotsoffGoal', 'away_ShotsonGoal',
       'home_TotalPasses', 'away_TotalPasses'],
      dtype='object')

### Rows with null values

In [687]:
matches_stats[matches_stats.isna().any(axis=1)]

,winners,match_id,home_id,away_id,odd_1,odd_2,odd_x,home_id_2611,home_id_2612,home_id_2613,...,away_id_2646,away_id_2654,home_BallPossession,home_ShotsoffGoal,home_ShotsonGoal,away_BallPossession,away_ShotsoffGoal,away_ShotsonGoal,home_TotalPasses,away_TotalPasses
53,2613,164854,2615,2613,0.474291,0.269460,0.323341,0,0,0,...,0,0,49%,6.0,3.0,51%,5.0,3.0,NaN,NaN
191,2646,237422,2630,2646,0.306290,0.398287,0.292708,0,0,0,...,1,0,42%,2.0,3.0,58%,2.0,6.0,NaN,NaN
192,2626,237423,2626,2620,0.494321,0.261518,0.284728,0,0,0,...,0,0,55%,1.0,10.0,45%,4.0,5.0,NaN,NaN
193,2630,267786,2630,2620,0.262871,0.540113,0.269595,0,0,0,...,0,0,42%,4.0,2.0,58%,4.0,0.0,NaN,NaN
310,2623,224540,2641,2623,0.416034,0.305573,0.264240,0,0,0,...,0,0,66%,7.0,5.0,34%,6.0,2.0,NaN,NaN


# Rakip Datalarını da ekle!

In [688]:
def get_team_rates(match_id, team_id, order, new = False):
    
    if new == False:
        index = matches_gb[matches_gb['match_id'] == match_id ].index[0]
        games_played = matches_gb.loc[:index,].copy()
    if new == True:
        games_played = matches_gb.copy()
    
    
    
    games_played = games_played[(games_played.away_id == team_id) | (games_played.home_id == team_id)].reset_index(drop=True)
    
    home_games = matches_stats[(matches_stats.home_id == team_id)]
    away_games = matches_stats[(matches_stats.away_id == team_id)]
    
    total_win_rate = games_played.winners.tolist().count(str(team_id)) / len(games_played['winners'])
    home_win_rate = games_played[games_played['home_id'] == team_id].winners.tolist().count(str(team_id)) / len(games_played[games_played['home_id'] == team_id]['winners'])
    away_win_rate = games_played[games_played['away_id'] == team_id].winners.tolist().count(str(team_id)) / len(games_played[games_played['away_id'] == team_id]['winners'])
    
    last6_games = games_played.tail(6).copy()
    last6_home_games = last6_games[last6_games['home_id'] == team_id ] # Home Games in Last 6 matches
    last6_away_games = last6_games[last6_games['away_id'] == team_id ] # Away Games in Last 6 matches
    
    oldest_match = last6_games.match_id.values[0]
    
    team_pts = calculate_team_points(oldest_match)  # Points of all teams until the given match.
    
    goals_conceded_home = sum(last6_home_games.away_score.tolist())
    goals_conceded_away = sum(last6_home_games.home_score.tolist())
    
    total_goals_conceded = goals_conceded_away + goals_conceded_home
    
    mean_conceded_goals = statistics.mean(last6_home_games.away_score.tolist() + last6_home_games.home_score.tolist())
    
    
    home_goals = last6_home_games.home_score.tolist()
    home_goals = np.asarray(home_goals)
    
    away_goals = last6_away_games.away_score.tolist()
    away_goals = np.asarray(away_goals)
    

    home_points = list()
    for i in last6_home_games.away_id.tolist():
        home_points.append(team_pts[team_pts['teams'] == str(i)]['points'].tolist()[0])

    home_points = np.asarray(home_points)
    home_score_points = home_points*home_goals
    
    away_points = list()
    for i in last6_away_games.home_id.tolist():
        away_points.append(team_pts[team_pts['teams'] == str(i)]['points'].tolist()[0])
        
    away_points = np.asarray(away_points)
    away_score_points = away_points*away_goals
    
    score_points = sum(home_score_points) + sum(away_score_points)                      # Score Points
    
    stdev_of_goals = statistics.stdev(list(home_goals) + list(away_goals))              # Standart Deviation of Goals
    
    if new == False:
        games_played = matches_stats.loc[:index,].copy()
    else:
        games_played = matches_stats.copy()
    
    home_possessions = home_games['home_BallPossession'].tail(6).values.tolist()
    summation = 0
    for j in home_possessions:
        summation += int(j.strip('%'))
    
    home_mean_possession = summation / len(home_possessions)   # Mean Ball Possession in  Last 6 Home Matches
        
    away_possessions = away_games['away_BallPossession'].tail(6).values.tolist()
    summation = 0
    for j in away_possessions:
        summation += int(j.strip('%'))
    
    away_mean_possession = summation / len(away_possessions)   # Mean Ball Possession in  Last 6 Away Matches
    
    
    home_shots_on = home_games['home_ShotsonGoal'].tail(6).values.tolist()
    mean_home_shots_on = statistics.mean(home_shots_on)
    away_shots_on = away_games['away_ShotsonGoal'].tail(6).values.tolist()
    mean_away_shots_on = statistics.mean(away_shots_on)
    
    home_shots_off = home_games['home_ShotsoffGoal'].tail(6).values.tolist()
    away_shots_off = away_games['away_ShotsoffGoal'].tail(6).values.tolist()
        
    shots_rate = sum(np.asarray(home_shots_on + away_shots_on)) / sum(np.asarray(home_shots_on + home_shots_off + away_shots_on + away_shots_off))
    
    
    
    if order == 1:
        value = {'total_win_rate': total_win_rate, 'home_win_rate': home_win_rate, 'away_win_rate': away_win_rate, 'away_goals': sum(away_goals), 'home_goals': sum(home_goals), 'score_points': score_points, 'goals_conceded': total_goals_conceded, 'mean_conceded': mean_conceded_goals, 'stdev_goals':stdev_of_goals, 'h_mean_poss': home_mean_possession, 'a_mean_poss': away_mean_possession, 'h_mean_shotson':  mean_home_shots_on, 'a_mean_shotson':mean_away_shots_on, 'shots_rate':shots_rate }
    else:
        value = {'total_win_rate2': total_win_rate, 'home_win_rate2': home_win_rate, 'away_win_rate2': away_win_rate, 'away_goals2': sum(away_goals), 'home_goals2': sum(home_goals), 'score_points2': score_points, 'goals_conceded2': total_goals_conceded, 'mean_conceded2': mean_conceded_goals, 'stdev_goals2':stdev_of_goals, 'h_mean_poss2': home_mean_possession, 'a_mean_poss2': away_mean_possession, 'h_mean_shotson2':  mean_home_shots_on, 'a_mean_shotson2':mean_away_shots_on, 'shots_rate2':shots_rate }
    
    return value
    
# Data to Return:  Total Win Rate, Home Win Rate, Away Win Rate, H. Goals, A. Goals, Score Points, Goals Conceded, Mean Goals Conceded, Stdev of Goals, H Mean Poss, A Mean Poss, mean H.ShotsOn, mean A.ShotsOn, ShotsRate
# Score Points: is calculated by multiplying each scored goal quantities for each match with the Point of the Opponent Team.
    
    

**Örnek**

In [ ]:
games_played = matches_stats.loc[:334,].copy()
games_played = games_played[(games_played.away_id == 2626) | (games_played.home_id == 2626)].reset_index(drop=True)
home_games = games_played[games_played['home_id'] == 2626]

#int(home_games['home_BallPossession'].values.tolist()[0].strip('%'))

home_games.away_id.tolist()

<hr>

### Here we create the dataframe for training:

In [689]:
matches_stats.head()

,winners,match_id,home_id,away_id,odd_1,odd_2,odd_x,home_id_2611,home_id_2612,home_id_2613,...,away_id_2646,away_id_2654,home_BallPossession,home_ShotsoffGoal,home_ShotsonGoal,away_BallPossession,away_ShotsoffGoal,away_ShotsonGoal,home_TotalPasses,away_TotalPasses
0,Tie,150842,2616,2612,0.194175,0.568182,0.270270,0,0,0,...,0,0,69%,7.0,4.0,31%,3.0,1.0,688.0,311.0
1,2626,150844,2626,2627,0.343249,0.410959,0.303030,0,0,0,...,0,0,65%,6.0,5.0,35%,1.0,1.0,703.0,380.0
2,2626,156848,2626,2615,0.370430,0.375756,0.340690,0,0,0,...,0,0,73%,4.0,6.0,27%,2.0,1.0,786.0,287.0
3,2611,156847,2611,2623,0.588560,0.202962,0.289960,1,0,0,...,0,0,38%,3.0,3.0,62%,6.0,0.0,304.0,498.0
4,2619,156845,2619,2629,0.432232,0.305744,0.345338,0,0,0,...,0,0,58%,11.0,9.0,42%,2.0,0.0,519.0,356.0


In [690]:
 matches_stats.tail()

,winners,match_id,home_id,away_id,odd_1,odd_2,odd_x,home_id_2611,home_id_2612,home_id_2613,...,away_id_2646,away_id_2654,home_BallPossession,home_ShotsoffGoal,home_ShotsonGoal,away_BallPossession,away_ShotsoffGoal,away_ShotsonGoal,home_TotalPasses,away_TotalPasses
343,2621,273252,2615,2621,0.160383,0.604368,0.213310,0,0,0,...,0,0,25%,1.0,0.0,75%,4.0,8.0,271.0,821.0
344,2627,273255,2626,2627,0.770984,0.101384,0.177350,0,0,0,...,0,0,72%,6.0,5.0,28%,2.0,7.0,659.0,268.0
345,2611,273251,2632,2611,0.250307,0.446748,0.259619,0,0,0,...,0,0,40%,9.0,2.0,60%,8.0,8.0,300.0,463.0
346,2654,273257,2641,2654,0.346245,0.416715,0.289072,0,0,0,...,0,1,58%,5.0,4.0,42%,5.0,3.0,570.0,413.0
347,Tie,273253,2613,2646,0.376615,0.356073,0.319251,0,0,1,...,1,0,61%,5.0,4.0,39%,3.0,5.0,569.0,353.0


In [691]:
rows = list()
for i in matches_stats.match_id.index:
    match_id = matches_stats.at[i,'match_id']
    home_id = matches_stats.at[i,'home_id']
    away_id = matches_stats.at[i,'away_id']
    #print(home_id)
    
    
    
    winner = {'winner': 0}
    win = {'home_win' : 0 , 'away_win' : 0, 'tie' : 0}
    
    if matches_stats.at[i,'winners'] == str(home_id):
        winner['winner'] = 1
        win['home_win'] = 1
    elif matches_stats.at[i, 'winners'] == str(away_id):
        winner['winner'] = 3
        win['away_win'] = 1
    else:
        winner['winner'] = 2
        win['tie'] = 1
        
    temp = {'match_id':match_id, 'home_id':home_id, 'away_id':away_id, 'odd_1':matches_stats.at[i,'odd_1'], 'odd_2':matches_stats.at[i,'odd_2'], 'odd_x':matches_stats.at[i,'odd_x']}
    
    winner.update(temp)
    
    dt = get_team_rates(match_id = match_id, team_id = home_id, order = 1)
    winner.update(dt)
    
    dt2 = get_team_rates(match_id = match_id, team_id = away_id , order = 2)
    winner.update(dt2)
    
    rows.append(winner)
    
    
df  = pd.DataFrame(rows)
df.head(10)

,winner,match_id,home_id,away_id,odd_1,odd_2,odd_x,total_win_rate,home_win_rate,away_win_rate,...,home_goals2,score_points2,goals_conceded2,mean_conceded2,stdev_goals2,h_mean_poss2,a_mean_poss2,h_mean_shotson2,a_mean_shotson2,shots_rate2
0,2,150842,2616,2612,0.194175,0.568182,0.270270,0.488889,0.538462,0.421053,...,5.0,32.206897,6.0,1.500000,1.211060,47.000000,48.166667,6.666667,4.166667,0.541667
1,1,150844,2626,2627,0.343249,0.410959,0.303030,0.769231,0.857143,0.666667,...,5.0,48.706897,8.0,2.000000,0.752773,48.166667,50.666667,5.333333,5.500000,0.511811
2,1,156848,2626,2615,0.370430,0.375756,0.340690,0.780488,0.863636,0.684211,...,4.0,27.103448,9.0,1.125000,0.632456,41.333333,54.000000,3.500000,3.000000,0.493671
3,1,156847,2611,2623,0.588560,0.202962,0.289960,0.324324,0.400000,0.235294,...,0.0,9.982759,7.0,1.750000,0.836660,44.500000,40.000000,3.833333,2.166667,0.382979
4,1,156845,2619,2629,0.432232,0.305744,0.345338,0.200000,0.285714,0.105263,...,1.0,9.051724,7.0,1.750000,0.836660,37.833333,39.166667,5.333333,2.833333,0.475728
5,2,156850,2614,2627,0.567334,0.212101,0.302934,0.142857,0.157895,0.130435,...,2.0,40.655172,3.0,0.750000,0.836660,48.166667,50.666667,5.333333,5.500000,0.511811
6,1,157536,2617,2628,0.306538,0.461496,0.302019,0.512195,0.695652,0.277778,...,3.0,52.810345,5.0,1.250000,1.264911,64.333333,50.000000,4.000000,3.833333,0.427273
7,1,157537,2621,2612,0.774154,0.103383,0.192520,0.658537,0.684211,0.636364,...,5.0,32.206897,6.0,1.500000,1.211060,47.000000,48.166667,6.666667,4.166667,0.541667
8,1,158697,2613,2619,0.877893,0.057569,0.114520,0.300000,0.454545,0.111111,...,4.0,25.551724,7.0,1.166667,0.894427,41.666667,45.666667,3.166667,2.166667,0.432432
9,3,158699,2623,2626,0.614704,0.181418,0.250912,0.297297,0.421053,0.166667,...,12.0,66.482759,15.0,2.500000,1.722401,68.333333,69.000000,5.500000,8.000000,0.503106


In [692]:
df = pd.merge(df,matches_stats , on =['match_id', 'home_id', 'away_id', 'odd_1', 'odd_2', 'odd_x']).drop(['winners','home_BallPossession', 'home_ShotsoffGoal', 'home_ShotsonGoal',
       'away_BallPossession', 'away_ShotsoffGoal', 'away_ShotsonGoal',
       'home_TotalPasses', 'away_TotalPasses' ], axis=1)

## One Hot Encoding for Team ID's

In [693]:
df['h_mean_poss'] = df['h_mean_poss']/100
df['h_mean_poss2'] = df['h_mean_poss2']/100
df['a_mean_poss'] = df['a_mean_poss']/100
df['a_mean_poss2'] = df['a_mean_poss2']/100
df.head()

,winner,match_id,home_id,away_id,odd_1,odd_2,odd_x,total_win_rate,home_win_rate,away_win_rate,...,away_id_2623,away_id_2626,away_id_2627,away_id_2628,away_id_2629,away_id_2630,away_id_2632,away_id_2641,away_id_2646,away_id_2654
0,2,150842,2616,2612,0.194175,0.568182,0.270270,0.488889,0.538462,0.421053,...,0,0,0,0,0,0,0,0,0,0
1,1,150844,2626,2627,0.343249,0.410959,0.303030,0.769231,0.857143,0.666667,...,0,0,1,0,0,0,0,0,0,0
2,1,156848,2626,2615,0.370430,0.375756,0.340690,0.780488,0.863636,0.684211,...,0,0,0,0,0,0,0,0,0,0
3,1,156847,2611,2623,0.588560,0.202962,0.289960,0.324324,0.400000,0.235294,...,1,0,0,0,0,0,0,0,0,0
4,1,156845,2619,2629,0.432232,0.305744,0.345338,0.200000,0.285714,0.105263,...,0,0,0,0,1,0,0,0,0,0


**Split the data into train and test:**

In [735]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,4:], df.iloc[:,[0]], test_size = 0.15)

In [736]:
y_reals = y_test.values.ravel()

### Logistic Regresion

In [737]:
"""from sklearn.preprocessing import MinMaxScaler 
sc_x = MinMaxScaler() 
x_train = sc_x.fit_transform(x_train)  
x_test = sc_x.transform(x_test)
"""
from sklearn.linear_model import LogisticRegression
logy = LogisticRegression(multi_class='multinomial' ,solver='newton-cg')
logy.fit(x_train,y_train.values.ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [738]:
def rps(p_home, p_draw, p_away, result):
    probs = [p_home, p_draw, p_away]
    result_p = [0,0,0]
    result_p[result-1] = 1
    rps_sum = 0
    for i in range(1,4):
        sum_p = 0
        for k in range(i):
            sum_p += probs[k]
        sum_r = 0
        for j in range(i):
            sum_r += result_p[j]
        rps_sum += (sum_p - sum_r)**2
    return rps_sum/2

In [739]:
y_pred = logy.predict_proba(x_test)
y_preds = logy.predict(x_test)
logy.score(x_test,y_reals)

0.6415094339622641

In [740]:
y_reals

array([2, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 3, 1, 1, 3, 2, 1, 2, 1, 2, 3,
       3, 1, 3, 2, 1, 3, 3, 3, 2, 2, 1, 1, 1, 1, 1, 3, 1, 1, 3, 3, 1, 1,
       1, 2, 1, 1, 3, 3, 1, 1, 1], dtype=int64)

In [741]:
y_preds

array([2, 3, 3, 1, 1, 3, 3, 1, 2, 3, 1, 3, 3, 1, 1, 3, 2, 1, 1, 1, 1, 1,
       3, 1, 1, 3, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 1, 3, 1, 1, 3, 3, 2, 2,
       3, 1, 1, 1, 1, 3, 1, 1, 1], dtype=int64)

In [742]:
y_pred  # P(Home) , P(Tie) , P(Away)

array([[1.30687520e-01, 5.98569698e-01, 2.70742781e-01],
       [2.74341774e-01, 1.29067390e-01, 5.96590837e-01],
       [7.13734445e-02, 2.06385120e-01, 7.22241436e-01],
       [9.05053580e-01, 6.90492650e-02, 2.58971545e-02],
       [7.77839681e-01, 7.10587370e-02, 1.51101582e-01],
       [2.77891913e-01, 1.24586032e-01, 5.97522055e-01],
       [7.03536712e-02, 5.41538004e-02, 8.75492528e-01],
       [4.50017200e-01, 2.89526748e-01, 2.60456052e-01],
       [4.49024617e-01, 5.10782997e-01, 4.01923856e-02],
       [3.16196375e-01, 2.95880757e-01, 3.87922868e-01],
       [9.91781945e-01, 8.20447209e-03, 1.35825335e-05],
       [2.08259705e-01, 1.49158967e-01, 6.42581329e-01],
       [1.74887735e-02, 9.70557542e-02, 8.85455472e-01],
       [9.61202012e-01, 3.86726109e-02, 1.25376984e-04],
       [5.62285295e-01, 3.79798178e-01, 5.79165273e-02],
       [1.20673075e-01, 2.94424473e-01, 5.84902452e-01],
       [8.83169422e-02, 7.02500985e-01, 2.09182073e-01],
       [5.65280859e-01, 2.38612

In [743]:
rpses = list()
for i in range(len(y_reals)):
    
    rpses.append(rps(y_pred[i][0], y_pred[i][1], y_pred[i][2], y_reals[i]))
    #print(rps(y_pred[i][0], y_pred[i][1], y_pred[i][2], y_reals[i]))
    
print(statistics.mean(rpses))

0.18983595621077537


### Upcoming Matches

Dataframe of the upcoming games would be:

In [744]:
coming_games.head()

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id,...,away_id_2623,away_id_2626,away_id_2627,away_id_2628,away_id_2629,away_id_2630,away_id_2632,away_id_2641,away_id_2646,away_id_2654
0,2617,2620,273260,West Ham,Arsenal,,,,,148,...,0,0,0,0,0,0,0,0,0,0
1,2623,2621,273266,Liverpool,Watford,,,,,148,...,1,0,0,0,0,0,0,0,0,0
2,2615,2616,273263,Chelsea,Bournemouth,,,,,148,...,0,0,0,0,0,0,0,0,0,0
3,2630,2629,273262,Burnley,Newcastle,,,,,148,...,0,0,0,0,0,1,0,0,0,0
4,2632,2654,273268,Sheffield Utd,Aston Villa,,,,,148,...,0,0,0,0,0,0,1,0,0,0


## Predicting Next Matches

We already have our dataframe named **coming_games**

In [745]:
results = str()
rows2 = list()
for i in range(len(coming_games.match_id.values.tolist())):
    match_id = coming_games.at[i,'match_id']
    home_id = coming_games.at[i,'home_id']
    away_id = coming_games.at[i,'away_id']
    
    temp2 = {'match_id':match_id, 'home_id':home_id, 'away_id':away_id, 'odd_1':matches_stats.at[i,'odd_1'], 'odd_2':matches_stats.at[i,'odd_2'], 'odd_x':matches_stats.at[i,'odd_x']}
    
    first = get_team_rates(match_id, home_id, order = 1, new = True)
    second = get_team_rates(match_id, away_id, order = 2, new = True)
    
    temp2.update(first)
    temp2.update(second)
    
    rows2.append(temp2)
    
df2  = pd.DataFrame(rows2)
   
df2.head()




,match_id,home_id,away_id,odd_1,odd_2,odd_x,total_win_rate,home_win_rate,away_win_rate,away_goals,...,home_goals2,score_points2,goals_conceded2,mean_conceded2,stdev_goals2,h_mean_poss2,a_mean_poss2,h_mean_shotson2,a_mean_shotson2,shots_rate2
0,273260,2620,2617,0.194175,0.568182,0.270270,0.246753,0.282051,0.210526,1.0,...,6.0,20.568966,13.0,1.625000,0.816497,54.833333,53.000000,5.166667,3.333333,0.447368
1,273266,2621,2623,0.343249,0.410959,0.303030,0.737500,0.815789,0.666667,7.0,...,1.0,10.827586,6.0,1.000000,0.816497,44.500000,40.000000,3.833333,2.166667,0.382979
2,273263,2616,2615,0.370430,0.375756,0.340690,0.477273,0.541667,0.400000,4.0,...,2.0,26.724138,7.0,1.166667,0.752773,41.333333,54.000000,3.500000,3.000000,0.493671
3,273262,2629,2630,0.588560,0.202962,0.289960,0.266667,0.324324,0.210526,3.0,...,6.0,41.724138,10.0,1.666667,0.983192,34.666667,30.000000,4.166667,3.166667,0.505747
4,273268,2654,2632,0.432232,0.305744,0.345338,0.333333,0.461538,0.214286,4.0,...,4.0,44.155172,10.0,1.666667,0.516398,51.333333,48.166667,4.666667,6.666667,0.511278


In [746]:
df2 = pd.merge(df2, coming_games, on = ['match_id', 'home_id', 'away_id']).drop(['home_name', 'away_name', 'home_score', 'away_score', 'home_half_score',
       'away_half_score', 'league_id'],axis=1)

In [747]:
df2['h_mean_poss'] = df2['h_mean_poss']/100
df2['h_mean_poss2'] = df2['h_mean_poss2']/100
df2['a_mean_poss'] = df2['a_mean_poss']/100
df2['a_mean_poss2'] = df2['a_mean_poss2']/100

In [ ]:
preds = logy.predict_proba(df2.iloc[:,3:])

In [756]:
coming_games.iloc[:,:5]

,away_id,home_id,match_id,home_name,away_name
0,2617,2620,273260,West Ham,Arsenal
1,2623,2621,273266,Liverpool,Watford
2,2615,2616,273263,Chelsea,Bournemouth
3,2630,2629,273262,Burnley,Newcastle
4,2632,2654,273268,Sheffield Utd,Aston Villa
5,2641,2611,273265,Leicester,Norwich
6,2620,2614,273269,Southampton,West Ham
7,2612,2627,273267,Manchester Utd,Everton
8,2628,2646,273270,Wolves,Tottenham
9,2626,2617,273261,Arsenal,Manchester City


In [757]:
preds # P(Home)    P(Tie)   P(Away)

results=str()
for i in range(len(preds)):
    results += f'{df2.match_id.values[i]},{preds[i][0]},{preds[i][2]},{preds[i][1]},'
    print(f'{df2.match_id.values[i]},{preds[i][0]},{preds[i][1]},{preds[i][2]}')

273260,0.4139092423827449,0.4724280581944951,0.11366269942275997
273266,0.9723372153170878,0.026944348866587243,0.0007184358163249788
273263,0.6803624106097039,0.2713686065010072,0.04826898288928896
273262,0.16791818097865524,0.08425398243248997,0.7478278365888548
273268,0.5725501387721832,0.04256316563242013,0.38488669559539673
273265,0.8509325184303914,0.09600419391240038,0.0530632876572082
273269,0.567012562213938,0.41013451842870696,0.02285291935735499
273267,0.6280721117271426,0.22645319330432825,0.14547469496852922
273270,0.6067467478145167,0.345429945492332,0.04782330669315124
273261,0.013751398221839716,0.005531878739613411,0.9807167230385467
273264,0.2058238278149386,0.5444930885489344,0.24968308363612693


In [758]:
results

'273260,0.4139092423827449,0.11366269942275997,0.4724280581944951,273266,0.9723372153170878,0.0007184358163249788,0.026944348866587243,273263,0.6803624106097039,0.04826898288928896,0.2713686065010072,273262,0.16791818097865524,0.7478278365888548,0.08425398243248997,273268,0.5725501387721832,0.38488669559539673,0.04256316563242013,273265,0.8509325184303914,0.0530632876572082,0.09600419391240038,273269,0.567012562213938,0.02285291935735499,0.41013451842870696,273267,0.6280721117271426,0.14547469496852922,0.22645319330432825,273270,0.6067467478145167,0.04782330669315124,0.345429945492332,273261,0.013751398221839716,0.9807167230385467,0.005531878739613411,273264,0.2058238278149386,0.24968308363612693,0.5444930885489344,'

### Results Like:   Match ID , P(Home) , P(Away) , P(Tie)

In [584]:
results

'273254,0.37365252568881546,0.24681482123323945,0.3949168633165657,273259,0.4241089065722088,0.3047904672227968,0.3139098765686054,273252,0.15133962522856426,0.23748080589362663,0.5844024681930056,273258,0.6446261622161904,0.208777550315566,0.16730908367169786,273255,0.5853324574643664,0.17770901217833723,0.20539800926258756,273251,0.4085744043022795,0.2465110325031709,0.3368710317433544,273256,0.5054537157642096,0.2713161718293473,0.24610166863847405,273257,0.3896632490821944,0.28017956185905435,0.32064660867051675,273253,0.3579553271963815,0.2766342078187854,0.35584247321821144,273260,0.3781219379789444,0.2509994251973582,0.36921483112947656,'

In [586]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [595]:
x_train.head()

,total_win_rate,home_win_rate,away_win_rate,away_goals,home_goals,score_points,goals_conceded,mean_conceded,stdev_goals,h_mean_poss,...,home_goals2,score_points2,goals_conceded2,mean_conceded2,stdev_goals2,h_mean_poss2,a_mean_poss2,h_mean_shotson2,a_mean_shotson2,shots_rate2
177,0.300000,0.250000,0.333333,6.0,4.0,16.315789,9.0,2.250000,1.032796,52.500000,...,7.0,15.578947,11.0,1.833333,0.752773,48.000000,50.500000,4.000000,2.333333,0.413043
274,0.776316,0.853659,0.685714,8.0,12.0,54.736842,14.0,2.333333,2.658320,67.833333,...,5.0,51.684211,11.0,1.833333,0.516398,48.000000,42.666667,2.500000,4.666667,0.443299
145,0.476923,0.555556,0.379310,0.0,8.0,30.894737,11.0,1.100000,1.211060,61.000000,...,3.0,18.368421,6.0,1.000000,0.816497,53.666667,50.166667,5.333333,4.666667,0.535714
92,0.666667,0.720000,0.620690,3.0,8.0,36.105263,12.0,2.000000,1.329160,56.166667,...,10.0,42.368421,16.0,2.000000,1.632993,41.500000,48.666667,4.666667,3.166667,0.522222
175,0.237288,0.281250,0.185185,2.0,5.0,28.631579,9.0,1.500000,1.169045,53.666667,...,7.0,34.421053,14.0,2.333333,1.224745,47.666667,41.666667,3.500000,4.500000,0.480000


(299, 28)

## DEEP LEARNİNG

In [91]:
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [92]:
x_train.shape

(266, 71)

In [151]:

model = Sequential()
model.add(Dense(71, input_dim=71, kernel_initializer = 'normal', activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(1, activation='softmax'))

model.compile(loss='mean_squared_error' ,optimizer='adam', metrics=['accuracy'])
    

'''estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, x_train, y_train, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))'''


model.fit(x_train, y_train, epochs=200, batch_size=10)
_, accuracy = model.evaluate(x_train, y_train)
predictions = model.predict_classes(x_test)

Epoch 1/200
266/266 [==============================] - ETA: 3s - loss: 1.0000 - accuracy: 0.60 - 0s 707us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 2/200
266/266 [==============================] - ETA: 0s - loss: 1.7000 - accuracy: 0.50 - 0s 180us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 3/200
266/266 [==============================] - ETA: 0s - loss: 1.2000 - accuracy: 0.70 - 0s 196us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 4/200
266/266 [==============================] - ETA: 0s - loss: 2.1000 - accuracy: 0.40 - 0s 165us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 5/200
266/266 [==============================] - ETA: 0s - loss: 2.6000 - accuracy: 0.20 - ETA: 0s - loss: 1.4462 - accuracy: 0.47 - 0s 226us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 6/200
266/266 [==============================] - ETA: 0s - loss: 1.7000 - accuracy: 0.50 - 0s 196us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 7/200
266/266 [==============================] - ETA: 0s - loss: 1.4000 - accuracy

266/266 [==============================] - ETA: 0s - loss: 1.1000 - accuracy: 0.50 - 0s 195us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 56/200
266/266 [==============================] - ETA: 0s - loss: 1.0000 - accuracy: 0.60 - 0s 180us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 57/200
266/266 [==============================] - ETA: 0s - loss: 1.1000 - accuracy: 0.50 - 0s 195us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 58/200
266/266 [==============================] - ETA: 0s - loss: 1.7000 - accuracy: 0.50 - 0s 155us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 59/200
266/266 [==============================] - ETA: 0s - loss: 1.1000 - accuracy: 0.50 - 0s 165us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 60/200
266/266 [==============================] - ETA: 0s - loss: 1.9000 - accuracy: 0.30 - 0s 165us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 61/200
266/266 [==============================] - ETA: 0s - loss: 1.4000 - accuracy: 0.50 - 0s 165us/step - loss: 1.4286 - accuracy

266/266 [==============================] - ETA: 0s - loss: 2.1000 - accuracy: 0.40 - 0s 180us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 111/200
266/266 [==============================] - ETA: 0s - loss: 0.9000 - accuracy: 0.70 - 0s 195us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 112/200
266/266 [==============================] - ETA: 0s - loss: 0.8000 - accuracy: 0.50 - 0s 165us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 113/200
266/266 [==============================] - ETA: 0s - loss: 1.0000 - accuracy: 0.60 - 0s 195us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 114/200
266/266 [==============================] - ETA: 0s - loss: 1.8000 - accuracy: 0.40 - 0s 180us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 115/200
266/266 [==============================] - ETA: 0s - loss: 2.3000 - accuracy: 0.20 - 0s 195us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 116/200
266/266 [==============================] - ETA: 0s - loss: 0.6000 - accuracy: 0.70 - 0s 195us/step - loss: 1.4286 - ac

Epoch 166/200
266/266 [==============================] - ETA: 0s - loss: 0.7000 - accuracy: 0.60 - 0s 165us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 167/200
266/266 [==============================] - ETA: 0s - loss: 1.4000 - accuracy: 0.50 - 0s 195us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 168/200
266/266 [==============================] - ETA: 0s - loss: 1.5000 - accuracy: 0.40 - 0s 195us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 169/200
266/266 [==============================] - ETA: 0s - loss: 1.5000 - accuracy: 0.40 - 0s 195us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 170/200
266/266 [==============================] - ETA: 0s - loss: 2.3000 - accuracy: 0.20 - 0s 165us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 171/200
266/266 [==============================] - ETA: 0s - loss: 1.3000 - accuracy: 0.60 - 0s 165us/step - loss: 1.4286 - accuracy: 0.4850
Epoch 172/200
266/266 [==============================] - ETA: 0s - loss: 1.4000 - accuracy: 0.50 - 0s 165us/step - los

In [152]:
predictions

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [64]:
predictions = model.predict(x_test)

NameError: name 'model' is not defined

In [662]:
predictions - y_test

,odd_1,odd_2,odd_x
173,0.134394,-0.050868,-0.084063
252,-0.008973,-0.000637,-0.034028
66,-0.030144,0.107880,0.019248
31,-0.107325,0.346146,0.007455
239,0.027441,0.006241,-0.006228
118,-0.111560,0.129872,-0.009941
316,0.064878,0.030037,-0.019468
37,-0.130890,0.125866,0.002884
220,-0.088637,0.070511,-0.009961
133,0.075755,-0.050608,-0.031211


In [663]:
mean_absolute_percentage_error(y_test, predictions)

odd_1    21.979367
odd_2    35.258671
odd_x    14.747657
dtype: float64

**Örnek**

In [534]:
first = get_team_rates(273236, 2630, order = 1, new = True)
second = get_team_rates(273236, 2626, order = 2, new = True)

first.update(second)

predicted_values = lr.predict(pd.DataFrame([first]))
mean_absolute_percentage_error(testtyy, predicted_values)

odd_1     6.988548
odd_2    31.318917
odd_x    20.312032
dtype: float64

In [514]:
predicted_values

array([[0.12001478, 0.59722681, 0.25328849]])

<hr>

In [341]:
matches_stats.head()

,match_id,winners,home_id,away_id,odd_1,odd_2,odd_x,home_BallPossession,home_ShotsoffGoal,home_ShotsonGoal,away_BallPossession,away_ShotsoffGoal,away_ShotsonGoal,home_TotalPasses,away_TotalPasses
0,150842,Tie,2616,2612,0.194175,0.568182,0.270270,69%,7.0,4.0,31%,3.0,1.0,688.0,311.0
1,150844,2626,2626,2627,0.343249,0.410959,0.303030,65%,6.0,5.0,35%,1.0,1.0,703.0,380.0
2,156848,2626,2626,2615,0.370430,0.375756,0.340690,73%,4.0,6.0,27%,2.0,1.0,786.0,287.0
3,156847,2611,2611,2623,0.588560,0.202962,0.289960,38%,3.0,3.0,62%,6.0,0.0,304.0,498.0
4,156845,2619,2619,2629,0.432232,0.305744,0.345338,58%,11.0,9.0,42%,2.0,0.0,519.0,356.0


In [289]:
merged.head()

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id,winners,odd_1,odd_2,odd_x
0,2612,2616,150842,Chelsea,Everton,0,0,0,0,148,Tie,0.194175,0.568182,0.270270
1,2627,2626,150844,Manchester City,Manchester Utd,3,1,1,0,148,2626,0.343249,0.410959,0.303030
2,2615,2626,156848,Manchester City,Bournemouth,3,1,1,1,148,2626,0.370430,0.375756,0.340690
3,2623,2611,156847,Leicester,Watford,2,0,2,0,148,2611,0.588560,0.202962,0.289960
4,2629,2619,156845,Crystal Palace,Burnley,2,0,1,0,148,2619,0.432232,0.305744,0.345338


In [239]:
merged.tail(10)

,away_id,home_id,match_id,home_name,away_name,home_score,away_score,home_half_score,away_half_score,league_id,winners,odd_1,odd_2,odd_x
318,2613,2627,224538,Manchester Utd,Brighton,3,1,2,0,148,2627,0.393417,0.168939,0.242085
319,2632,2646,224543,Wolves,Aston Villa,2,1,1,0,148,2646,0.506912,0.251371,0.283651
320,2626,2621,224537,Liverpool,Manchester City,3,1,2,0,148,2621,0.380521,0.270305,0.256530
321,2628,2620,273230,West Ham,Tottenham,2,3,0,2,148,2628,0.270584,0.519106,0.259018
322,2611,2613,273224,Brighton,Leicester,0,2,0,0,148,2611,0.275667,0.488490,0.284202
323,2641,2612,273226,Everton,Norwich,0,2,0,0,148,2641,0.719347,0.129386,0.199673
324,2646,2615,273223,Bournemouth,Wolves,1,2,0,2,148,2646,0.376366,0.371463,0.302872
325,2616,2626,273227,Manchester City,Chelsea,2,1,2,1,148,2626,0.706259,0.141358,0.202904
326,2627,2654,273228,Sheffield Utd,Manchester Utd,3,3,1,0,148,Tie,0.282177,0.459189,0.306843
327,2630,2632,273222,Aston Villa,Newcastle,2,0,2,0,148,2632,0.474385,0.281078,0.289531


In [472]:
goals = pd.read_csv('goals.csv')
goals.head()

,match_id,time,home_scorer,score,away_scorer
0,13327,30,Salah M.,1 - 1,NaN
1,13446,35,Colin M.,1 - 0,NaN
2,13446,56,NaN,1 - 1,Johnson D.
3,13446,60,NaN,1 - 2,Hugill J.
4,13446,67,NaN,1 - 3,Barkhuizen T.


In [ ]:
booking = pd.read_csv('booking.csv')
booking.head(30)